<br>

# Dateutil

O [dateutil](https://dateutil.readthedocs.io/) é um pacote que _"fornece extensões poderosas para o módulo datetime padrão, disponível em Python"_. Ele aborda questões muito mais gerais, para lidar com datas no python, e não é apenas um módulo para tratar de feriados.

No _site_ [Labix.org](http://labix.org/python-dateutil#head-470fa22b2db72000d7abe698a5783a46b0731b57), tem vários exemplos interessante sobre o uso de _Recurrence Rules_, uma função que pode ser usada para feriados.


In [ ]:
from datetime import date, datetime, timedelta

from dateutil import parser, rrule

import brazilian_holidays

<br>

---

## Recurrence Rules

O módulo `rrules` permite criar uma série de regras de horários recorrentes. Abaixo um exemlo de uma regra.


In [ ]:
list(
    rrule.rrule(
        rrule.DAILY,
        byweekday=[rrule.MO, rrule.TU, rrule.WE, rrule.TH, rrule.FR],
        count=10,
        dtstart=parser.parse('19970902T090000'),
    )
)

Exemplo com regras de minutos.

In [ ]:
list(
    rrule.rrule(
        rrule.MINUTELY,
        byhour=range(9, 17),
        byminute=(0, 30),
        # interval=30,
        byweekday=[rrule.MO, rrule.TU, rrule.WE, rrule.TH, rrule.FR],
        count=20,
        dtstart=date.today(),
    )
)

<br>

---

## Feriados nos EUA

Encontrei no repositório [adamJLev/holidays.py](https://gist.github.com/adamJLev/7535869), uma função que cria os feriados dos Estados Unidos utilizando `rrules`.

In [ ]:
def get_schedule_holidays_rrules():
    return [
        # New Years
        rrule.rrule(
            rrule.YEARLY, dtstart=datetime.now(), bymonth=1, bymonthday=1
        ),
        # Memorial
        rrule.rrule(
            rrule.YEARLY,
            dtstart=datetime.now(),
            bymonth=5,
            byweekday=rrule.MO(-1),
        ),
        # Independence
        rrule.rrule(
            rrule.YEARLY, dtstart=datetime.now(), bymonth=7, bymonthday=4
        ),
        # Thanksgiving
        rrule.rrule(
            rrule.YEARLY,
            dtstart=datetime.now(),
            bymonth=11,
            byweekday=rrule.TH(4),
        ),
        # Christmas
        rrule.rrule(
            rrule.YEARLY, dtstart=datetime.now(), bymonth=12, bymonthday=25
        ),
    ]

In [ ]:
get_schedule_holidays_rrules()

<br>

Pensava que poderia utilizar isso também para feriados brasileiros, porém ao me deparar com feriados de data móvel (páscoa e carnaval, por exemplo), desisti de utilizar o `rrules` para feriados. De qualquer forma, o pacote pode agregar muito nas análises de séries temporais, pela exclusão de feriados de intervalos sub-diários.

<br>

---

## Exclusão de Feriados

No artigo [StackOverFlow: **Datetime Python - Next Business Day**](https://stackoverflow.com/questions/9187215/datetime-python-next-business-day) descobri que dá pra definir feriados!!! E usar isso na biblioteca _dateutil_ para excluir das regras!!!



### Excluindo datas das `rrule`

Defini regras do `rrule` que eu desejaria obter intervalores de 30 minutos, entre as 9h e as 17h, e desejaria excluir algumas datas (ou "feriados") específicas.

In [ ]:
# Feriados
holidays = [
    datetime(2024, 3, 26, 11, 0, 0),
    datetime(2024, 3, 26, 11, 30, 0),
]

# Create a rule to recur every weekday starting today
r = rrule.rrule(
    rrule.MINUTELY,
    byhour=range(9, 17),
    byminute=(0, 30),
    byweekday=[rrule.MO, rrule.TU, rrule.WE, rrule.TH, rrule.FR],
    count=40,
    #dtstart=date.today(),
    dtstart=date(2024, 3, 24),
)

# Create a rruleset
rs = rrule.rruleset()

# Attach our rrule to it
rs.rrule(r)

# Add holidays as exclusion days
for holiday in holidays:
    rs.exdate(holiday)

# Results
list_intervals = list(rs)
list_intervals

<br>

Logo pensei: *posso usar o `brazilian-holidays` para criar uma lista de feriados!!!*

Contudo, infelizmente só dá certo se tiver também o horário definido.

<br>

----

### Excluindo Feriados Brasileiros de Séries Temporais

Pensava que, caso eu criasse uma lista de feriados, em formato `datetime`, eu conseguiria excluir os feriados de uma regra definida com o módulo `dateutil.rrule`.

Para isso criei a lista de feriados, utilizando o `brazilian_holidays`.




Pensei em ooutra coisa

In [ ]:
holidays = brazilian_holidays.Holidays(year=2024)
holidays.add_all()

holidays = holidays.create_list(tipo='date')

<br>

Inicialmente criamos todos os horários possíveis com o intervalo pretendido, para todos os feriados. Essa será a nossa tabela de exclusão.

In [ ]:
list_holidays_hours = []
for holiday in holidays:
    # Create a rule to recur every weekday starting today
    r = rrule.rrule(
        rrule.MINUTELY,
        byhour=range(9, 17),
        byminute=(0, 30),
        byweekday=[rrule.MO, rrule.TU, rrule.WE, rrule.TH, rrule.FR],
        dtstart=holiday,
        until=holiday + timedelta(days=1) # Tomorrow,
    )

    # Create a rruleset
    rs = rrule.rruleset()

    # Attach our rrule to it
    rs.rrule(r)

    # Results
    list_intervals = list(rs)
    list_holidays_hours.extend(list_intervals)
    
# Results
list_holidays_hours[:5]

<br>

Após isso recriados a regra, com a exclusão dos feridos.

In [ ]:
# Create a rule to recur every weekday starting today
r = rrule.rrule(
    rrule.MINUTELY,
    byhour=range(9, 17),
    byminute=(0, 30),
    byweekday=[rrule.MO, rrule.TU, rrule.WE, rrule.TH, rrule.FR],
    #count=50,    
    dtstart=date(2024, 3, 27),
    until=date(2024, 4, 2),
)

# Create a rruleset
rs = rrule.rruleset()

# Attach our rrule to it
rs.rrule(r)

# Add holidays as exclusion days
for holiday in list_holidays_hours:
    rs.exdate(holiday)

# Results
list_intervals = list(rs)
list_intervals

<br>

Note que o dia 28 e 29 de março de 2024 foram removidos, visto que são feriado definidos no `brazilian-holidays`.